# A Guide to Q&A using  Retrieval-Augmented Generation (RAG) with distributed local LLM embedding and generation

## Introduction
In this notebook, we'll demonstrate how to develop a context-aware question answering framework using distributed local LLM embedding and answer generation using Hugging Face models: [Phi-3-mini-4k-instruct](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) and [NV-Embed-v1](https://huggingface.co/bzantium/NV-Embed-v1). This notebook extending document Question and Answering demo to use only local models for scalability and acceleration. Question and Answering contect is based on NASA's Earth and Earth at Night e-books.    

We’ll cover the following key stages:

1. Load PDF documents using PyMUPDF library.
2. Use SynapseML to split the documents into chunks.
3. Generate chunk and user question embeddings using NV-Embed-V1 embedder
4. Using NVIDIA Rapids KNN find chunks related to user questions to define context for LLM answers
5. Using LLM Phi-3 from Microsoft and Tensor-RT GPU accelerator answer user questions using provided context

The demo was tested on NVIDIA A100 based Databricks Azure cluster with two workers based on Standard_NC24ads_A100_v4 using 13.3 LTS ML (includes Apache Spark 3.4.1, GPU, Scala 2.12) Databricks Runtime.


### Step 1: Define the notebook environment

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import fitz  # PyMuPDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from synapse.ml.featurize.text import PageSplitter
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, explode
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.functions import predict_batch_udf
from sentence_transformers import SentenceTransformer
import numpy as np
from spark_rapids_ml.knn import (
    ApproximateNearestNeighbors,
    ApproximateNearestNeighborsModel,
)
import pyspark.sql.functions as F
from pyspark.sql.functions import concat_ws, collect_list

### Step 2: Load the documents into a Spark DataFrame.

For this tutorial, we will be using NASA's [Earth](https://www.nasa.gov/sites/default/files/atoms/files/earth_book_2019_tagged.pdf) and [Earth at Night](https://www.nasa.gov/sites/default/files/atoms/files/earth_at_night_508.pdf) e-books. To load PDF documents into a Spark DataFrame, you can use the ```spark.read.format("binaryFile")``` method provided by Apache Spark.

In [0]:
document_path = "wasbs://publicwasb@mmlspark.blob.core.windows.net/NASAEarth"  # path to your document
df = spark.read.format("binaryFile").load(document_path).cache()

### Step 3: Read the document context and convert it from PDF to text using PyMUPDF library.

We utilize PyMUPDF library (fitz) to do PDF to Text conversion

In [0]:
# Define the function to extract text from binary PDF data
def extract_text_from_binary_pdf(binary_content):
    try:
        # Create a PyMuPDF document from the binary data
        doc = fitz.open(stream=binary_content, filetype="pdf")
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        return str(e)

# Register the function as a UDF
extract_text_udf = udf(extract_text_from_binary_pdf, StringType())

# Apply the UDF to extract text from the binary content
analyzed_df = df.withColumn("output_content", extract_text_udf(df["content"]))

We can split Spark DataFrame named ```analyzed_df``` in chunks to make book analysed context smaller (3000 - 4000 char) using the following code.

In [0]:
ps = (
    PageSplitter()
    .setInputCol("output_content")
    .setMaximumPageLength(4000)
    .setMinimumPageLength(3000)
    .setOutputCol("chunks")
)

splitted_df = ps.transform(analyzed_df)

In [0]:
# Each column contains many chunks for the same document as a vector.
# Explode will distribute and replicate the content of a vecor across multple rows
# Add id column

exploded_df = splitted_df.select("path", explode(col("chunks")).alias("chunk")).select("path", "chunk").withColumn("id", monotonically_increasing_id())

### Step 4: Generate Embeddings.

To produce embeddings for each chunk, we utilize NVIDIA NV-Embed-V1 embedder from Hugging Face

In [0]:
# Define a function to create the encode_udf with a custom query_prefix
def create_encode_udf(query_prefix):
    # Define a function to encode text in batches
    # def encode_text_batch(texts):
    def encode_text_batch():
        # Load the model inside the function
        model = SentenceTransformer('bzantium/NV-Embed-v1', trust_remote_code=True)
        model.max_seq_length = 4096
        model.tokenizer.padding_side = "right"
        
        def predict(inputs):
        
            output = model.encode(
                inputs.tolist(), prompt=query_prefix, normalize_embeddings=True)
            return output

        return predict

        # # Encode the texts in batch
        # embeddings = model.encode(inputs.tolist(), normalize_embeddings=True)
        # return [embedding.tolist() for embedding in embeddings]

    # Define the predict_batch_udf with the above function
    return predict_batch_udf(
        encode_text_batch,
        return_type=ArrayType(FloatType()),
        batch_size=1
    )

In [0]:
# Use it withhout query_prefix in this case
query_prefix = ""
encode_udf = create_encode_udf(query_prefix)

# Applying the UDF to a DataFrame chunk column
embeddings = exploded_df.withColumn("embeddings", encode_udf(col("chunk")))

### Step 5: Use chunk embeddings to create KNN search model to find chunks related to user query 

In [0]:
rapids_knn_model = (
    ApproximateNearestNeighbors(k=2)
    .setInputCol("embeddings")
    .setIdCol("id")
    .fit(embeddings)
)

### Step 6: Compose a Question.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

task_name_to_instruct = {"example": "Given a question, retrieve passages from the provided context that answer the question",}

query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "

encode_udf = create_encode_udf(query_prefix)

user_question = "What did the astronaut Edgar Mitchell call Earth?"
# Define schema explicitly
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("query", StringType(), True)
])

# Create DataFrame with id = 1 and the user query
temp_df = spark.createDataFrame([(1, user_question)], schema).cache()

# Apply the UDF to generate the embeddings
query_embeddings = temp_df.withColumn("embeddings", encode_udf(col("query")))

### Step 7: Find chunks with the closest context to the question using embeddings

In [0]:
(_, _, knn_df) = rapids_knn_model.kneighbors(query_embeddings.select("id", "embeddings"))

In [0]:
# Add text to the results
result_df = (
        knn_df.withColumn(
            "zipped", F.explode(F.arrays_zip(F.col("indices"), F.col("distances")))
        )
        .select(
            F.col("query_id"),
            F.col("zipped.indices").alias("id"),
            F.col("zipped.distances").alias("distance"),
        )
        .join(embeddings, on="id", how="inner")
        .select("query_id", "id", "chunk", "distance")
    )

In [0]:
# Concatenate all strings in the 'combined_text' column across all question related chunks
concatenated_text = result_df.agg(concat_ws(" ", collect_list("chunk")).alias("concatenated_text")).collect()[0]["concatenated_text"]


### Step 8: Respond to a User’s Question using microsoft/Phi-3-mini-4k-instruct LLM from Hugging Face

In [0]:
from tensorrt_llm import LLM, SamplingParams, BuildConfig

# Put model in global if we want to reuse it
global llm

if 'llm' in globals() and llm is not None:
    print("Model is already loaded.")
else:
    print("Model is not loaded.")
    
    # Extend model input sizes
    build_config = BuildConfig()
    build_config.plugin_config.context_fmha = True
    build_config.max_input_len = 5120
    build_config.max_seq_len = 5632

    llm = LLM(model="microsoft/Phi-3-mini-4k-instruct", build_config=build_config)    
    
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

context = concatenated_text
query = "What did the astronaut Edgar Mitchell call Earth?"

prompt = f"""
context: {context}
Answer the question based only on the context above. Without multiple choices. If the
information to answer the question is not present in the given context then reply "I don't know".
My Question: {query}
What is your Answer? """

outputs = llm.generate(prompt, sampling_params)

### Step 9: Print LLM results

In [0]:
# Mocking the custom classes if they are not already defined
class CompletionOutput:
    def __init__(self, index, text, token_ids, cumulative_logprob=None, logprobs=None):
        self.index = index
        self.text = text
        self.token_ids = token_ids
        self.cumulative_logprob = cumulative_logprob
        self.logprobs = logprobs

class RequestOutput:
    def __init__(self, request_id, prompt, prompt_token_ids, outputs, finished):
        self.request_id = request_id
        self.prompt = prompt
        self.prompt_token_ids = prompt_token_ids
        self.outputs = outputs
        self.finished = finished

output_text = outputs.outputs[0].text

# Split the text by '\n'
split_text = output_text.split('\n')

for item in split_text:
    if len(item) > 10:
        # Split the item at the colon and take the part after it
        result = item.split(':', 1)[-1].strip()
        print("Answer: " + result)
        break

We can now wrap up the Q&A journey by asking a question and checking the answer. You will see that Edgar Mitchell called Earth "a sparkling blue and white jewel"!